In [ ]:
from geodesic.tesseract import Job, AssetSpec, Step, Container, StepInput, StepOutput, TemporalBinning

In [ ]:
job = Job(
    alias="Test Job",
    description=f"VIIRS Nighttime Test",
    global_properties=GlobalProperties(
        pixel_size=[2000.0, 2000.0],                           
        chip_size=2000
    ),
    bbox=bbox,                                       
    bbox_epsg=4326,                                 
    output_epsg=3857,
    asset_specs=[
        AssetSpec(
            name="viirs",                       
            dataset="viirs-monthly",           
            datetime=[
                '2020-07-01T00:00:00',              
                '2021-07-01T00:00:00'
            ],
            assets=['monthly_average_radiance'],                
            pixel_type=np.float32,                   
            resample='cubicspline',                 
            input_no_data=['nan'],                                  
            temporal_binning=TemporalBinning(       
                user=User(bins=bins)
            )
        )
    ],
    steps=[
        Step(
            name="test",
            type="model",
            container=Container(
                repository="gcr.io/double-catfish-291717",
                image="tesseract-identity-model",
                tag="v0.0.1"
            ),
            gpu=True,
            inputs=[
                StepInput(
                    asset_name="viirs",
                    chunk_shape=(12, 1, 39, 36),
                    type="tensor",
                )
            ],
            outputs=[
                StepOutput(
                    asset_name="identity",
                    chunk_shape=(12, 1, 39, 36),
                    type="tensor",
                    pixel_dtype=np.float32,
                    fill_value="nan"
                ),
                StepOutput(
                    asset_name="count",
                    chunk_shape=(1, 1, 1, 1),
                    type="tensor",
                    pixel_dtype=np.float32,
                    fill_value="nan"
                )
            ]
        ),
    ],
    workers=1
)